In [1]:
import ipyparallel as ipp
from scipy import stats
import numpy as np
import time

In [7]:
c = ipp.Client()
#create a directview
dview = c[:]

In [8]:
c.ids

[0, 1, 2]

In [9]:
from sklearn import datasets
iris = datasets.load_iris()
train = iris.data
target = iris.target

In [10]:
#sending date to engines
c[0]['X'] = train
c[1]['X'] = train
c[2]['X'] = train

In [144]:
def random_init_w(k_cluster,X):
    describe = stats.describe(X)
    w = np.random.normal(loc = describe[2], scale =describe[3], size = (k_cluster,len(X[0])))
    
    return w
    

In [11]:
#Algorithm 2 - line 7, retrieve w and c from aggregator 
def wc_from_eng(engine):
    '''
    Input:
        parameter of w recieved from engines may be matrices
    Output:
        reshaped w as vector
    '''
    w = []
    c = []
    for i in range(len(engine.ids)):
        w.append(engine[i]['km.get_coef()'])
        c.append(engine[i]['km.Est_Resource()']) 
    # reshape w from matrix to vector
    w = np.array(w)
    c = np.array(c)
    w = w.reshape(w.shape[0],1,-1)
    return w,c

In [12]:
#retrieve belta and grad to aggregator #Algorithm 2 - line 14
def bg_from_eng(engine):
    '''
    Input:
        parameter of gradient recieved from engines may be matrices
    Output:
        reshaped gradient as vector
    '''
    belta = []
    grad = []
    for i in range(len(engine.ids)):
        belta.append(engine[i]['km.belta'])
        grad.append(engine[i]['km.grad_t0']) 
    grad = np.array(grad)
    belta = np.array(belta)
    grad = grad.reshape(grad.shape[0],1,-1)
    
    return belta,grad

In [73]:
#Algorithm 2 - line 8, global parameter updata according to (5)
data_size = np.array([50,50,50])
def global_update(w_local,data_size):
    '''Aggregator parameter updating rules: mean'''
    '''
    Input: Matrix with shape = (# of engines, # of parameters)
    Output: Matrix with shape = (# of engines, # of parameters)
    '''
    temp = 0
    for i in range(len(w_local)):
        temp = temp + w_local[i] * data_size[i]
    return np.array(temp/data_size.sum())        

In [74]:
#Algorithm 2 - line 15
def belta_update(belta_local,data_size):
    '''Aggregator belta updating rules: mean'''
    '''
    Input: Matrix with shape = (# of engines, # of parameters)
    Output: Matrix with shape = (# of engines, # of parameters)
    '''
    temp = 0
    for i in range(len(belta_local)):
        temp = temp + np.array(belta_local[i]) * data_size[i]
    return temp/data_size.sum()

In [15]:
#Algorithm 2 - line 16
def grad_update(grad_local,data_size):
    temp = 0
    for i in range(len(grad_local)):
        temp = temp + np.array(grad_local[i]) * data_size[i]
    return temp/data_size.sum()

def delta_update(local_grad, grad_aggregator, data_size):
    delta_local = []
    local_grad = np.array(local_grad) 
    grad_aggregator = np.array(grad_aggregator)
    for item in local_grad:
        temp_delta = np.linalg.norm(item - grad_aggregator)
        delta_local.append(temp_delta)
    return belta_update(delta_local,data_size)

In [16]:
def grad_kmeans(w, X, cluster_id):
    shape = w.shape
    grad = np.zeros(shape = shape)
    for m in range(shape[0]):
        for n in range(shape[1]):
            grad[m,n] = w[m,n]*len(cluster_id[m]) - X[cluster_id[m],n].sum()
    return grad

In [17]:
from sklearn.metrics.pairwise import euclidean_distances
def eu_distance(X, centroid):
    return euclidean_distances(X,centroid)    

In [18]:
def partition(distance_matrix):
    cluster = [[] for i in range(distance_matrix.shape[1])]
    for i in range(len(distance_matrix)):
        cluster_label = np.argmin(distance_matrix[i])
        cluster[cluster_label].append(i)
    return cluster

In [19]:
#send torque and w to engines from aggregator
def snd_to_eng(w,torque,engine):
    for i in range(len(engine.ids)):
        engine[i]['w_aggregator'] = w
        engine[i]['torque_aggregator'] = torque[i]

In [20]:
#Algorithm 2 - line 17, binary search for new torque
def G(torque, delta, belta, eta, phi):
    h = delta* (pow((eta*belta + 1),torque)-1)/belta - eta*delta*torque
    G = torque*(eta*(1-belta*eta/2)-phi*h/torque)
    return G

def binary_search(torque,delta, belta, gamma,phi,eta = 0.0001):
    upper_bound = int(gamma * torque)
    G_list = []
    for i in range(upper_bound):
        torque_try = i + 1
        G_list.append(G(torque_try, delta, belta, eta, phi))
    torque_star = np.argmax(np.array(G_list)) + 1
    return torque_star, G_list

In [114]:
import numpy as np
import time
class Kmeans:

    def __init__(self):
        self.t = 0
        self.t_0 = 0
        self.grad = 0
        self.learning_rate = 0.001
        self.torque = 0
        self.belta = 0
        self.resource = 0
        self.grad_t0 = 0
        self.test = 0
        self.history = []
        self.w = 0
        self.w_hat = 0
        self.w_t0 = 0
    def Rec_from_Eng(self, w_global, torque_global):
        '''
        w_global: parameter recieved from aggregator, has a shape of (k, features)
        
        '''
        self.w_t0 = w_global
        self.w_hat = w_global
        self.torque = torque_global
    def Snd_to_Agg(self):
        if self.t_0 > 0:
            return w,self.resource, self.belta, self.grad_t0
        else:
            return w,self.resource
    
    def Est_Resource(self):
        return self.resource

    def get_coef(self):
        return self.w
    
    def set_coef(self, w_global):
        self.w = w_global
        
    def Est_Belta(self ,X):
        #estimate clusters given global parameters
        distance_matrix = euclidean_distances(X, self.w_hat)
        cluster_id = partition(distance_matrix)
        grad_global_parameter = grad_kmeans(self.w_hat, X, cluster_id)
        self.grad_t0 = grad_global_parameter
        self.belta = np.linalg.norm(self.grad - grad_global_parameter)/np.linalg.norm(self.w - self.w_hat)
        
    def time_record(self):
        self.t_0 = self.t
        
    def fit(self, X):
        
        self.w = self.w_hat #updata local w by aggregator 
        
        count = 0
        tic = time.time()
        
        for i in range(self.torque): 
            self.t += 1
            distance_matrix = euclidean_distances(X, self.w)
            cluster_id = partition(distance_matrix)
            grad = grad_kmeans(self.w, X, cluster_id)
            self.w = self.w - self.learning_rate * grad
            
            count += 1
            if count < self.torque:
                self.w_hat = self.w
            elif count == self.torque:
                self.grad = grad  #self.grad save the gradient(t) when t has a global aggregation
            
            
        toc = time.time()
        self.resource = toc - tic
        
        return self
    

In [121]:
km = Kmeans()
dview.push(dict(Kmeans = Kmeans,partition = partition,grad_kmeans = grad_kmeans))
#sending LinearRegression object to engines
dview['km'] = km

In [145]:
import time 
t = 0
s = 0
b = 0 # aggregator consumption
R = 5
gamma = 10
phi = 0.02
w_init = random_init_w(3,train)
w_aggregator = w_init
torque_aggregator = [1,1,1]
stop = False
loss_hist = []

In [123]:
while True:
    tic = time.time()
    snd_to_eng(w_aggregator,torque_aggregator,c)
    t_0 = t
    t = t + torque_aggregator[0]
    dview.execute("""
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import time    
km.Rec_from_Eng(w_aggregator, torque_aggregator)
km.time_record()
if km.t > 0:
    km.Est_Belta(X)
km.fit(X)
    """)
    
    
    w_local, c_local = wc_from_eng(c)
    #calculate local consumption per iteration
    c_per = np.array(c_local)/torque_aggregator[0]
    w_aggregator = global_update(w_local, data_size)
    w_aggregator = w_aggregator.reshape(w_1_origin.shape)
    if stop:
        w_final = w_aggregator
        break
    #c_local.sum() equal to c*t
    s = s + np.array(c_local).sum()+ b
    
    if t_0 > 0:
        belta_local, grad_local = bg_from_eng(c)
        belta_aggregator = belta_update(belta_local,data_size)
        grad_aggregator = grad_update(grad_local,data_size)
        delta_aggregator = delta_update(grad_local, grad_aggregator, data_size)
        torque_update, G_list = binary_search(torque_aggregator[0], delta_aggregator, belta_aggregator, gamma, phi)
        torque_aggregator = [torque_update for i in range(len(c.ids))]
        print('New torque is:',torque_aggregator[0])
    
    toc = time.time()
    b = toc - tic
    temp = s + torque_aggregator[0]*c_per.sum() + b
    if temp >= R:
        torque_max = (R-b-s)/c_per.sum()
        G_list = np.array(G_list)
        G_min = G_list.min()
        for i, item in enumerate(G_list):
            if item >= torque_max:
                itme = G_min
        torque_updata = np.argmax(G_list) + 1
        torque_aggregator = [torque_update for i in range(len(c.ids))]
        stop = True 
    

New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1


In [118]:
w_aggregator.reshape(w_1_origin.shape)

array([[6.11747488, 2.99056493, 1.83303575, 1.01004811],
       [6.82024913, 3.22791872, 3.88144444, 1.80048304],
       [5.9986321 , 3.14435461, 3.60250955, 1.04922358]])

In [124]:
w_final

array([[5.54803849, 3.20955061, 1.6439695 , 0.6175825 ],
       [6.65052297, 3.06145011, 4.78259964, 1.83980675],
       [5.85005757, 2.97163109, 3.73637196, 1.11325422]])

In [120]:
dview['km.belta']

[8172596372655776.0, 8172596372655776.0, 8172596372655776.0]

In [36]:
dview.execute('''
import numpy as np
km.fit(X)

''')

<AsyncResult: execute>

In [127]:
w_1_origin

array([[5.23437157, 3.20839712, 5.93536268, 0.74439649],
       [5.66023644, 3.23406749, 4.96111899, 0.42790574],
       [5.04225614, 2.8031247 , 1.76407928, 0.4692056 ]])

In [24]:
from scipy import stats

In [25]:
a = stats.describe(train)

In [26]:
w_1_origin = np.random.normal(loc = a[2], scale = a[3], size = (3,4))

In [72]:
w_1_origin.shape

(3, 4)

In [63]:
aaa = np.arange(10)
mean = 0
for i in range(10000):
    grad = (np.array([mean for i in range(10)])-aaa).sum()
    mean = mean - 0.001*grad


In [136]:
s = np.array([925,725,725,625])

In [138]:
b = []
for i in range(len(s)):
    b.append(844.26*s[i]/s.sum())

In [139]:
b

[260.3135, 204.0295, 204.0295, 175.8875]

In [141]:
150+201+260


611

In [142]:
1500-611

889

In [143]:
500/4

125.0

In [146]:
w_init

array([[7.25792917, 2.83116628, 0.44922719, 1.25068524],
       [5.98822726, 2.87296436, 2.55375724, 1.43567045],
       [6.49868252, 3.06840254, 8.38735151, 0.03575358]])

In [145]:
889+76

965

In [146]:
527-76

451